<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_Lang_Chain_Routers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In LLM routers, we would give an instruction to LLM to how to answer different requests. Then we would send an input to the LLM and it would answer according to the instruction.**

**note:you can just copy these codes and just change the priliminary prompts (here are beginner template or expert template) as well as prompt info to make the LLM to be routed**

In [1]:
!pip install langchain
!pip install openai
!pip install wikipedia
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the 

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain

In [3]:
import os
openAI_API_Key=os.environ['openAI_API_Key']

In [4]:
beginner_template=''' you are a physics teacher who is really focused begineers and explaining complex concepts simple to understand terms.
you assume no prior knowledge. Here is your question: \n {input}'''

In [5]:
expert_template=''' you are a professor who explains physics topics to advanced audience members. you can assume anyone who answers has a PHD
in physics. Here is your question: \n{input}'''

In [6]:
prompt_info= [ {'name':'begineer physics',
    'description':' Answer basic physics questions',
    'template':beginner_template,}, { 'name':'Advanced physics',
    'description':' Answer advanced physics questions',
    'template':expert_template,}
]




In [7]:
llm=ChatOpenAI(openai_api_key=openAI_API_Key)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
destination_chains={}
for P_info in prompt_info:
  name=P_info['name']
  prompt_template=P_info['template']
  prompt=ChatPromptTemplate.from_template(template=prompt_template)
  chain=LLMChain(llm=llm, prompt=prompt)
  destination_chains[name]=chain


In [9]:
default_prompt=ChatPromptTemplate.from_template('{input}')
default_chain=LLMChain(llm=llm, prompt=default_prompt)

In [10]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [11]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
destination = [ f"{p['name']}:{p['description']}" for p in prompt_info]

In [13]:
destination

['begineer physics: Answer basic physics questions',
 'Advanced physics: Answer advanced physics questions']

In [14]:
destination_str="\n".join(destination)

In [15]:
print(destination_str)

begineer physics: Answer basic physics questions
Advanced physics: Answer advanced physics questions


In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [20]:
router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

In [21]:
router_template

'Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.\n\n<< FORMATTING >>\nReturn a markdown code snippet with a JSON object formatted to look like:\n```json\n{{\n    "destination": string \\ name of the prompt to use or "DEFAULT"\n    "next_inputs": string \\ a potentially modified version of the original input\n}}\n```\n\nREMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.\nREMEMBER: "next_inputs" can just be the original input if you don\'t think any modifications are needed.\n\n<< CANDIDATE PROMPTS >>\nbegineer physics: Answer basic physics questions\nAdvanced physics: Answer advance

In [23]:
router_prompt=PromptTemplate(template=router_template, input_variables=['input'], output_parser=RouterOutputParser())

In [24]:
from langchain.chains.router import MultiPromptChain

In [25]:
router_chain=LLMRouterChain.from_llm(llm, router_prompt)

In [29]:
chain=MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

In [30]:
chain.run('how does magnet work?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
begineer physics: {'input': 'how does magnet work?'}
> Finished chain.


'Great question! Let\'s dive into the fascinating world of magnets. \n\nMagnets are special objects that have the ability to attract or repel certain materials, like iron or steel. They work based on a property called magnetism. \n\nMagnetism is a force that exists in some materials, like iron or certain metals. Inside these materials, there are tiny particles called atoms, which are the building blocks of everything around us. \n\nNow, inside each atom, there are even tinier particles called electrons. These electrons have a property called "spin," which is like a spinning motion. When electrons spin in the same direction, they create a tiny magnetic field. Normally, in most materials, these tiny magnetic fields cancel each other out, so we don\'t notice any magnetic effects. \n\nHowever, in certain materials, like iron, the electrons can align their spins and create a larger magnetic field. When many atoms in a material have their magnetic fields aligned, it forms a larger magnetic f

In [32]:
chain.run('please explain feynman diagrams')



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


Advanced physics: {'input': 'please explain feynman diagrams'}
> Finished chain.


"Feynman diagrams are a powerful visual representation tool used in quantum field theory to depict particle interactions and calculate the probability of those interactions occurring. They were developed by physicist Richard Feynman in the 1940s and have since become an integral part of theoretical physics.\n\nAt its core, a Feynman diagram is a graphical depiction of various particles, their interactions, and their corresponding mathematical expressions. It provides a pictorial representation of the mathematical formulas used to describe the behavior of elementary particles in quantum field theory.\n\nIn a Feynman diagram, particles are represented by lines, or more precisely, propagators. These lines can be either straight or wavy, depending on the type of particle they represent. Straight lines typically depict elementary particles such as electrons, quarks, or photons, while wavy lines represent force-carrying particles, also known as gauge bosons.\n\nThe interactions between these